In [98]:
import serial
import re
import math
import sys

In [2]:
LF = "\x0A"
CR = "\x0D"
FF = "\x0C"
ESC = "\x1B"
CAN = "\x18"
SP = "\x20"
GS = "\x1D"
NULL = chr(0)

In [82]:
connection = serial.Serial(port="/dev/ttyUSB0",baudrate=19200)

In [83]:
w = connection.write

def cut():
    w(LF+LF+LF+LF+ESC+'i'+LF)

In [84]:
connection.write('\x0A')

1

In [125]:
cut()

###Impression d'images
####GS * n1 n2 [ d ] n1 x n2 x 8
*cf page 50*


**[Function]** 
Defining the download bit image


**[Code]** 
* <1D>H<2A>H<n1><n2> [< d >] n1 x n2 x 8

**[Range]** 
* 1≤n1≤255
* 1≤n2≤48
* n1 x n2 ≤1536

**[Outline]**
* Defines download bit images of the number of dots specified by “n1” and “n2”.
* The numbers of dots are n1   8 in horizontal direction and n2   8 in vertical direction.
* ”d” indicates bit image data.
* Once defined, the download bit image remains effective until it is redefined, ESC @,
ESC &, GS (A, or FS q, is executed, or power is turned OFF.

**[Caution]** 
* Relations between the bit image data and the dots defined are shown below.
* With this command executed, the defined content of a downloaded character is
cleared.

**[See Also]**
* GS /


**[Sample Program]**
```
    GOSUB IMG
    LPRINT CHR$(&H1D);"/"; CHR$(0);
    LPRINT CHR$(&H1D);"/"; CHR$(1);
    LPRINT CHR$(&H1D);"/"; CHR$(2);
    LPRINT CHR$(&H1D);"/"; CHR$(3);
    END
    
    IMG:
    n1=10:n2=5
    LPRINT CHR$(&H1D);"*";
    LPRINT CHR$(n1); CHR$(n2);
    FOR J=1 TO n1*8
    FOR I=1 TO n2
    LPRINT CHR$(J);
    NEXT I
    NEXT J
    RETURN
```

In [34]:
n1=1
n2=1

w(GS+'*')
w(chr(n1))
w(chr(n2))
for k in range(4*n1):
    for l in range(n2):
        w("\xFA")
        w("\xF0")

###Impression de l'image

GS+H

Modes : 
* 0,48
* 1,49
* 2,50
* 3,51


In [39]:
w("n1=%s, n2=%s"%(n1,n2) + LF)
for k in range(0,4):
    w("%s"%k+LF)
    w(GS+"/"+chr(k)+LF)

cut()

6

### Printing of raster bit image

**[Code]**  ``<1D>H<76>H<30>H<m><xL><xH><yL><yH> [<d>] k``


**[Range]**
* 0≤m≤3, 48≤m≤51, 
* 0≤xL≤255, 
* 0≤xH≤255,
* 0≤yL≤255, 
* 0≤yH≤8
* 0≤d≤255,
* k = (xL + xH x 256) x (yL + yH x 256), however, k ≠ 0


**[Outline]** Prints raster bit images in mode “m”.
* xL, xH specify the number of data in horizontal direction of the bit image to (xL + xH
x256) bytes.
* yL, yH specify the number of data in vertical direction of the bit image to (yL + yH x
256) bytes.

In [86]:
def startRaster(mode,xL,xH,yL,yH):
    w(GS+'v0') #Printing of raster bit imag
    w(chr(mode)) #Print mode (0:normal, 1:double width, 2:double height, 3: quadruple size)
    w(chr(xL)) #xL
    w(chr(xH)) #xH
    w(chr(yL)) #yL
    w(chr(yH)) #yH

In [95]:
startRaster(3,2,0,5,0)
# 2 lignes noires
w('\xFF')
w('\xFF')
# 2lignes mi noires, mi blanches
w('\x0F')
w('\x0F')
# 2 lignes mi blanches, mi noires
w('\xF0')
w('\xF0')

# grille d'1px sur deux en decalé sur 4 lignes
w('\xAA')
w('\x55')
w('\xAA')
w('\x55')

cut()

In [93]:
startRaster(3,1,0,2,0)
w('\xAA')
w('\xAA')
cut()

In [123]:
def printPBM(filename,mode=3):
    with open(filename,'r') as f:
        while True:
            header = f.readline().strip()
            if header.startswith('#'):
               continue
            elif header == 'P1':
               assert False, 'ASCII format not supported'
            elif header == 'P4':
               break
            else:
               assert False, 'Bad format !'


        rows, cols = 0, 0
        while True:
            header = f.readline().strip()
            if header.startswith('#'):
               continue
            else:
               match = re.match('^(\d+) (\d+)$',header)
               if match is None:
                   assert False, 'Bad size'
               else:
                   cols, rows = match.groups()
                   break
        rows, cols = int(rows), int(cols)
        assert (rows, cols) != (0, 0)
        colbytes = int(math.ceil(cols/8.0))

        xL,xH = colbytes%256, int(math.floor(colbytes/256.0))
        yL,yH = rows%256, int(math.floor(rows/256.0))
        
        startRaster(mode,xL,xH,yL,yH)
        for n in range(0,rows):
            for m in range(0,colbytes):
                mychar = f.read(1)
                #sys.stdout.write('%02X' % ord(mychar))
                w(mychar)

In [126]:
w("Bienvenue au Carrefour Numerique")
w(LF)
printPBM('/opt/pbm/cerveau.pbm',0)
cut()